In [1]:
from text_processing import *
from constants import *

In [2]:
file_selector = is_2017_in_file

In [3]:
judgements_files = filter(file_selector, os.listdir(json_data_dir))
texts = chain.from_iterable(map(lambda f: judgements_raw(f,json_data_dir), judgements_files))
texts = islice(texts,2000)
filtered = filter(filter_judgements, texts)

In [4]:
with_categories = map(lambda j: WithCategory(j, map_category(j).label), filtered)
cleaned = map(
    lambda j: Line(j.judgement['id'], clean_text(j.judgement['textContent'], common_words), j.category),
    with_categories)
cleaned  = list(cleaned)

In [5]:
tuples = map(lambda l: (l.words,l.category),cleaned)
listed = list(filter(lambda tup: tup[1] in ['civil', 'criminal', 'economic', 'insurance'],tuples))

x_train, x_test, y_train, y_test = get_train_test_data(listed)



In [9]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words = common_words)),
    ('clf', OneVsRestClassifier(LinearSVC())),
])

pipeline.fit(x_train, y_train)
predictions = pipeline.predict(x_test)

print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.88      0.97      0.92       167
          1       1.00      0.94      0.97        79
          2       0.86      0.20      0.32        30
          3       1.00      1.00      1.00        78

avg / total       0.93      0.90      0.90       354



In [ ]:
cat_id = dict(map(lambda line: (line.id, line.category), cleaned))
tagged_categories = list(tagged(cat_id))